In [271]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import pickle
from statistics import mean

In [272]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer, MWETokenizer
from nltk.util import ngrams
import re
import string
from nltk.corpus import stopwords
from nltk.chunk import ne_chunk
set(stopwords.words('english'))
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [273]:
got_stopwords = ['[spoilers]', '[deleted]', 'nan', '[removed]', '[spoiler]', 'spoilers', 'deleted', 
                 'removed', 'spoiler', '[All Spoilers]', 'all spoilers', '[NO SPOILER]', 'no spoiler', '[SPOLER]', 'spoler']

In [274]:
def cleaning_pipeline(text):
    cleaner_text = " ".join([word for word in text.split() if word.lower() not in got_stopwords])
    return cleaner_text

In [275]:
got_reddit = pickle.load( open( "got_reddit.pkl", "rb" ) )
asoiaf_reddit = pickle.load( open( "asoiaf_reddit.pkl", "rb" ) )
freefolk_reddit = pickle.load( open( "freefolk_reddit.pkl", "rb" ) )

In [276]:
print(freefolk_reddit.shape)
print(asoiaf_reddit.shape)
print(got_reddit.shape)

(139768, 2)
(24743, 2)
(133485, 2)


In [277]:
freefolk_reddit['sentiment'] = ''
asoiaf_reddit['sentiment'] = ''
got_reddit['sentiment'] = ''

freefolk_reddit['sentiment_compound'] = ''
asoiaf_reddit['sentiment_compound'] = ''
got_reddit['sentiment_compound'] = ''

freefolk_reddit['sentiment_sum'] = ''
asoiaf_reddit['sentiment_sum'] = ''
got_reddit['sentiment_sum'] = ''

freefolk_reddit['sentiment_mean'] = ''
asoiaf_reddit['sentiment_mean'] = ''
got_reddit['sentiment_mean'] = ''

In [278]:
counts_free = freefolk_reddit['body'].value_counts()
freefolk_reddit = freefolk_reddit[~freefolk_reddit['body'].isin(counts_free[counts_free > 1].index)]

counts_a = asoiaf_reddit['body'].value_counts()
asoiaf_reddit = asoiaf_reddit[~asoiaf_reddit['body'].isin(counts_a[counts_a > 1].index)]

counts_g = got_reddit['body'].value_counts()
got_reddit = got_reddit[~got_reddit['body'].isin(counts_g[counts_g > 1].index)]

In [279]:
print(freefolk_reddit.shape)
print(asoiaf_reddit.shape)
print(got_reddit.shape)

(130794, 6)
(24309, 6)
(129195, 6)


In [266]:
#pickle.dump(freefolk_reddit, open("freefolk_reddit_after_removing_spam.pkl", "wb" ) )
#pickle.dump(asoiaf_reddit, open("soiaf_reddit_after_removing_spam.pkl", "wb" ) )
#pickle.dump(got_reddit, open("got_reddit_after_removing_spam.pkl", "wb" ) )

In [280]:
sid = SentimentIntensityAnalyzer()

def sentiment(df):
    for i, j in df.iterrows(): 
        text = df['body'][i]
        cleaned_text = cleaning_pipeline(text)
        df['body'][i] = cleaned_text
        sentences = tokenize.sent_tokenize(cleaned_text)
        sentiment = []
        for sentence in sentences:
            ss = sid.polarity_scores(sentence)
            sentiment.append(ss)
        df['sentiment'][i] = sentiment

In [281]:
def sentiment_compound_func(df):
    for i, j in df.iterrows(): 
        text = df['sentiment'][i]
        sentiment_list = []
        for item in range(len(text)):
            sentiment = text[item]['compound']
            sentiment_list.append(sentiment)
            df['sentiment_compound'][i] = sentiment_list

In [282]:
def sentiment_sum_func(df):
    for i, j in df.iterrows(): 
        df['sentiment_sum'][i] = sum(df['sentiment_compound'][i])

In [283]:
def sentiment_mean_func(df):
    for i, j in df.iterrows(): 
        df['sentiment_mean'][i] = mean(list(df['sentiment_compound'][i]))

In [204]:
sentiment(freefolk_reddit)
sentiment(asoiaf_reddit)
sentiment(got_reddit)

sentiment_compound_func(freefolk_reddit)
sentiment_compound_func(asoiaf_reddit)
sentiment_compound_func(got_reddit)

sentiment_sum_func(freefolk_reddit)
sentiment_sum_func(asoiaf_reddit)
sentiment_sum_func(got_reddit)

sentiment_mean_func(freefolk_reddit)
sentiment_mean_func(asoiaf_reddit)
sentiment_meanfunc(got_reddit)

pickle.dump(freefolk_reddit, open("freefolk_reddit_final.pkl", "wb" ) )
pickle.dump(asoiaf_reddit, open("soiaf_reddit_final.pkl", "wb" ) )
pickle.dump(got_reddit, open("got_reddit_final.pkl", "wb" ) )